In [251]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [252]:
NVH_metrics = ['6208 Cage Failure', 'CH/Input Bearing Race Defect', 'CH/Input Bearing Ball Defect',
						'CH Inter Bearing Race Defect',
						'CH Inter Bearing Ball Defect',
						'Input Gear Clicking',
						'Intermediate Gear Clicking',
						'Output Gear Clicking',
						'Intermediate Gear Clicking Negative',
						'Intermediate Gear Clicking Positive',
						'Input Gear Clicking Positive',
						'Input Gear Clicking Negative',
						'Output Gear Clicking Negative',
						'Output Gear Clicking Positive', 
                        'Motor 3rd Ord Avg', 
                        'Growl 4-6']

# To find dyno performance
## Count unique Serials
This is the total number of units tested

## Find unique failures
How many unique serials with metricispass = 0

## Out of these serial failures, how many of them passed within the threshold time?
Compare the failing metric to the next occurance of the passing metric
Is it within the threshold time? If so, it's a retest pass. If not, it's a rework pass
If there's no next passing metric? True Failure, pending rework

## How to do?
testID and isTestPass can be used

Group the runs per df['testID'] and compare the times between the runs. 
If metricispass = 1 > metricispass = 0, < retestWindow_min = 15 min: It's a retest
If metricispass = 1 > metricispass = 0, > retestWindow_min = 15 min: It's a rework
If metricispass = 0 doesn't exist: Pending successful rework
Would need to account for breaks and lunches for retestWindow_min. Refer to the excel sheet you made for the megapack harrness guys


In [253]:
df_raw = pd.read_csv('datasets\Week23_DynoSummary.csv')
df_raw.head()

,testID,isTestPass,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek
0,253173,1,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:43:10.000,6,3,2024,6,43,6208 Cage Failure,1,49.188820,23
1,253173,1,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:43:10.000,6,3,2024,6,43,Max,1,429.695618,23
2,253173,1,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:43:10.000,6,3,2024,6,43,Torque Inverter Side,1,146.263199,23
3,253174,1,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:47:59.000,6,3,2024,6,47,6208 Cage Failure,1,55.186283,23
4,253174,1,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:47:59.000,6,3,2024,6,47,Max,1,555.106079,23


In [254]:
summary = pd.Series({'FirstPasses':0,
                     'DLE':0,
                           'Retest Success':0,
                           'ReworkedPasses':0,
                           'Pending Rework / Unsuccessful Rework':0,
                           'Machine Health':0
                           })

In [255]:
df = df_raw.copy()
#df.groupby('FX_Thing')['metricispass'].sum().min()
#if a test has 46 metricispass, it's a first time pass
#dropping Golden units
golden_units = ['P1025598-00-U:ST13D0013400', 'P1025598-00-U:ST17A1021978']
df = df[~df['FX_Thing'].isin(golden_units)]
retestTime = 20


In [256]:
#key and datetime
df['thing+metricName'] = df['FX_Thing']+'+'+df['metricName']
df['testTime'] = pd.to_datetime(df['testTime'])

In [257]:
df['part'].nunique()

109

In [258]:
#Finding all the first time passes. Every unit with 46 metricispass sum is a first time pass
sum_metricispass = df.groupby('FX_Thing')['metricispass'].sum()
firstPasses = sum_metricispass[sum_metricispass == 46].reset_index()['FX_Thing']
summary['FirstPasses'] = len(firstPasses)

In [259]:
#getting all the failures and the unique failing serials
df_allFailures = df[df['metricispass'] == 0].copy()
df_allFailures.head()

#count of unique failures
failing_serials = df_allFailures['FX_Thing'].drop_duplicates()
len(failing_serials)



10

In [260]:
#anything with only passes, but doesn't have 46 runs, is a DLE
DLEs = df['FX_Thing'].nunique() - len(firstPasses) - len(failing_serials)
summary['DLE'] = DLEs

In [261]:
#DLEs
uniqueSerials = df.drop_duplicates(subset = 'FX_Thing')
uniqueSerials[(~uniqueSerials['FX_Thing'].isin(failing_serials)) & (~uniqueSerials['FX_Thing'].isin(firstPasses))]

,testID,isTestPass,part,FX_Thing,partnumber,testTime,testMonth,testDay,testYear,testHour,testMinute,metricName,metricispass,metricValue,yearWeek,thing+metricName
0,253173,1,P1025598-01-U-ST13L0030717R2,P1025598-01-U:ST13L0030717R2,1025598-01-U,2024-06-03 06:43:10,6,3,2024,6,43,6208 Cage Failure,1,49.188820,23,P1025598-01-U:ST13L0030717R2+6208 Cage Failure
4831,253538,1,P1025598-01-U-ST14J0065840R2,P1025598-01-U:ST14J0065840R2,1025598-01-U,2024-06-06 05:31:35,6,6,2024,5,31,6208 Cage Failure,1,56.606480,23,P1025598-01-U:ST14J0065840R2+6208 Cage Failure
5015,253550,1,P1025598-00-U-ST16L1021579,P1025598-00-U:ST16L1021579,1025598-00-U,2024-06-06 06:01:30,6,6,2024,6,1,6208 Cage Failure,1,47.550797,23,P1025598-00-U:ST16L1021579+6208 Cage Failure


In [262]:
df_failures_and_Passes = df[df['thing+metricName'].isin(df_allFailures['thing+metricName'].unique())]

In [263]:
failures_and_passes_summary = df_failures_and_Passes.drop_duplicates(['part', 'metricName'])[['part', 'metricName', 'thing+metricName']]
failures_and_passes_summary 

,part,metricName,thing+metricName
109,P1025598-00-U-ST13D0013387,CH Inter Bearing Race Defect,P1025598-00-U:ST13D0013387+CH Inter Bearing Ra...
128,P1025598-00-U-ST13D0013387,Intermediate Gear Clicking,P1025598-00-U:ST13D0013387+Intermediate Gear C...
135,P1025598-00-U-ST13D0013387,Intermediate Gear Clicking Negative,P1025598-00-U:ST13D0013387+Intermediate Gear C...
383,P1025598-01-U-ST15C0086377,CH/Input Bearing Race Defect,P1025598-01-U:ST15C0086377+CH/Input Bearing Ra...
838,P1025598-01-U-ST15A0078219,Motor 3rd Ord Avg,P1025598-01-U:ST15A0078219+Motor 3rd Ord Avg
1804,P1025598-00-U-ST13D0013460R1,Motor 3rd Ord Avg,P1025598-00-U:ST13D0013460R1+Motor 3rd Ord Avg
1806,P1025598-00-U-ST13D0013460R1,Growl 4-6,P1025598-00-U:ST13D0013460R1+Growl 4-6
2349,P1025598-01-U-ST16H1013287,6208 Cage Failure,P1025598-01-U:ST16H1013287+6208 Cage Failure
2363,P1025598-01-U-ST16H1013287,CH Inter Bearing Race Defect,P1025598-01-U:ST16H1013287+CH Inter Bearing Ra...
2366,P1025598-01-U-ST16H1013287,Inv Gear Tack Skipping,P1025598-01-U:ST16H1013287+Inv Gear Tack Skipping


# Is there a passing test?


In [264]:
def isPassTest(part):
    df_tmp = df_failures_and_Passes[(df_failures_and_Passes['part'] == part) & (df_failures_and_Passes['isTestPass'] == 1)]
    if df_tmp.shape[0] == 0:
        return False
    else:
        return True

In [265]:
failures_and_passes_summary['PassingTest?'] = failures_and_passes_summary['part'].apply(isPassTest)

In [266]:
failures_and_passes_summary

,part,metricName,thing+metricName,PassingTest?
109,P1025598-00-U-ST13D0013387,CH Inter Bearing Race Defect,P1025598-00-U:ST13D0013387+CH Inter Bearing Ra...,True
128,P1025598-00-U-ST13D0013387,Intermediate Gear Clicking,P1025598-00-U:ST13D0013387+Intermediate Gear C...,True
135,P1025598-00-U-ST13D0013387,Intermediate Gear Clicking Negative,P1025598-00-U:ST13D0013387+Intermediate Gear C...,True
383,P1025598-01-U-ST15C0086377,CH/Input Bearing Race Defect,P1025598-01-U:ST15C0086377+CH/Input Bearing Ra...,False
838,P1025598-01-U-ST15A0078219,Motor 3rd Ord Avg,P1025598-01-U:ST15A0078219+Motor 3rd Ord Avg,True
1804,P1025598-00-U-ST13D0013460R1,Motor 3rd Ord Avg,P1025598-00-U:ST13D0013460R1+Motor 3rd Ord Avg,True
1806,P1025598-00-U-ST13D0013460R1,Growl 4-6,P1025598-00-U:ST13D0013460R1+Growl 4-6,True
2349,P1025598-01-U-ST16H1013287,6208 Cage Failure,P1025598-01-U:ST16H1013287+6208 Cage Failure,True
2363,P1025598-01-U-ST16H1013287,CH Inter Bearing Race Defect,P1025598-01-U:ST16H1013287+CH Inter Bearing Ra...,True
2366,P1025598-01-U-ST16H1013287,Inv Gear Tack Skipping,P1025598-01-U:ST16H1013287+Inv Gear Tack Skipping,True


In [267]:
def calculateTime(part): #If there is a successful rework, find the time between the latest success and the latest failure
    df_test = df_failures_and_Passes[df_failures_and_Passes['part'] == part].drop_duplicates(['testID', 'isTestPass']).reset_index()
    if 1 in df_test['isTestPass'].values:
        passingLoc = df_test[df_test['isTestPass'] == 1].index
        comparison_result = df_test.loc[passingLoc[0], 'testTime'] - df_test.loc[passingLoc[0] - 1, 'testTime']
        return comparison_result
    else: return np.nan

In [268]:
#cleaningTable
failures_and_passes_summary['TimeToPass'] = failures_and_passes_summary['part'].apply(calculateTime)
failures_and_passes_summary['TimeToPass_minutes'] = failures_and_passes_summary['TimeToPass'].apply(lambda x: x.total_seconds()/60)
failures_and_passes_summary['retestSuccess?'] = failures_and_passes_summary['TimeToPass_minutes'].apply(lambda x: True if x < retestTime else False)
failures_and_passes_summary['MachineHealth?'] = failures_and_passes_summary['metricName'].apply(lambda x: False if x in NVH_metrics else True)
failures_and_passes_summary

,part,metricName,thing+metricName,PassingTest?,TimeToPass,TimeToPass_minutes,retestSuccess?,MachineHealth?
109,P1025598-00-U-ST13D0013387,CH Inter Bearing Race Defect,P1025598-00-U:ST13D0013387+CH Inter Bearing Ra...,True,0 days 00:19:33,19.550000,True,False
128,P1025598-00-U-ST13D0013387,Intermediate Gear Clicking,P1025598-00-U:ST13D0013387+Intermediate Gear C...,True,0 days 00:19:33,19.550000,True,False
135,P1025598-00-U-ST13D0013387,Intermediate Gear Clicking Negative,P1025598-00-U:ST13D0013387+Intermediate Gear C...,True,0 days 00:19:33,19.550000,True,False
383,P1025598-01-U-ST15C0086377,CH/Input Bearing Race Defect,P1025598-01-U:ST15C0086377+CH/Input Bearing Ra...,False,NaT,NaN,False,False
838,P1025598-01-U-ST15A0078219,Motor 3rd Ord Avg,P1025598-01-U:ST15A0078219+Motor 3rd Ord Avg,True,0 days 00:07:10,7.166667,True,False
1804,P1025598-00-U-ST13D0013460R1,Motor 3rd Ord Avg,P1025598-00-U:ST13D0013460R1+Motor 3rd Ord Avg,True,0 days 00:06:37,6.616667,True,False
1806,P1025598-00-U-ST13D0013460R1,Growl 4-6,P1025598-00-U:ST13D0013460R1+Growl 4-6,True,0 days 00:06:37,6.616667,True,False
2349,P1025598-01-U-ST16H1013287,6208 Cage Failure,P1025598-01-U:ST16H1013287+6208 Cage Failure,True,2 days 04:42:58,3162.966667,False,False
2363,P1025598-01-U-ST16H1013287,CH Inter Bearing Race Defect,P1025598-01-U:ST16H1013287+CH Inter Bearing Ra...,True,2 days 04:42:58,3162.966667,False,False
2366,P1025598-01-U-ST16H1013287,Inv Gear Tack Skipping,P1025598-01-U:ST16H1013287+Inv Gear Tack Skipping,True,2 days 04:42:58,3162.966667,False,True


In [269]:
summary['Machine Health'] = failures_and_passes_summary[failures_and_passes_summary['MachineHealth?'] == True]['part'].nunique()
fp_summary_withoutMH = failures_and_passes_summary[failures_and_passes_summary['MachineHealth?'] == False].copy()

In [270]:
summary['Retest Success']=fp_summary_withoutMH[fp_summary_withoutMH['retestSuccess?'] == True]['part'].nunique()

In [271]:
summary['ReworkedPasses'] = fp_summary_withoutMH[(fp_summary_withoutMH['retestSuccess?'] == False) & (fp_summary_withoutMH['PassingTest?'] == True)]['part'].nunique()

In [272]:
summary['Pending Rework / Unsuccessful Rework'] = fp_summary_withoutMH[fp_summary_withoutMH['PassingTest?'] == False]['part'].nunique()

In [287]:
def failDisposition(part):
    if part['MachineHealth?'] == True:
        return 'Machine Health Failure'
    elif part['retestSuccess?'] == True:
        return 'retest Success'
    elif part['PassingTest?'] == True:
        return 'Reworked Pass'
    else:
        return 'Pending Rework / Unsuccessful Rework'

In [293]:
failures_and_passes_summary.to_csv('failures.csv')
summary.to_csv('summary.csv')